# Building RAG Chatbots for Technical Documentation

## Table of contents



## Introduction 

This project ...


## Environment Setup

// to be done

## Load the document



## Split the document

## Generate and store the embeddings



In [ ]:
# install langchain
%pip install -qU langchain langchain-community langchain-chroma langchain-text-splitters unstructured sentence_transformers

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredHTMLLoader


file_path = "document.html"

loader = UnstructuredHTMLLoader(file_path)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
docs = loader.load_and_split(text_splitter)

print(f"{docs[1].metadata}\n")
print(docs[1].page_content)
print(len(docs))



{'source': 'document.html', 'start_index': 763}

After consulting the Committee of the Regions,

Acting in accordance with the ordinary legislative procedure (2),

Whereas:

(1) Council Directive 90/385/EEC ( 3 ) and Council Directive 93/42/EEC ( 4 ) constitute the Union regulatory framework for medical devices, other than in vitro diagnostic medical devices. However, a fundamental revision of those Directives is needed to establish a robust, transparent, predictable and sustainable regulatory framework for medical devices which ensures a high level of safety and health whilst supporting innovation.
853


In [8]:
# Generate and store the embeddings
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)



ModuleNotFoundError: No module named 'langchain_huggingface'